In [2]:
import pandas as pd

books = pd.read_csv("data/books.csv")
print(books.columns)

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')


In [3]:
books2=books[["book_id","goodreads_book_id","title","authors","average_rating","ratings_count"]].sort_values(["average_rating","ratings_count"],ascending=False)

# Calcul de la note pondérée (weighted rating)

$$ wr = ( \frac{v}{v+m} \cdot R ) + ( \frac{m}{v+m} \cdot C ) $$

- $v$ le nombre de votes par livre
- $m$ le minimum de votes requis pour être dans la liste
- $R$ la note moyenne du livre
- $C$ la note moyenne de tous les livres

In [5]:
# Calcul de C, à partir des moyennes de notes
C = books2["average_rating"].mean()

# Calcul de m, à partire du 50ème quantile
m = books2["ratings_count"].quantile(0.75)
books90 = books2[books2["ratings_count"]>=m]

# Fonction qui calcule la note pondérée
def weighted_rating(x, m=m, C=C):

    v = x["ratings_count"]
    R = x["average_rating"]

    return ( v/(v+m) *R)+(m/(v+m)*C)

books90["weighted_rating"] = books90.apply(weighted_rating, axis=1)

C:\Users\bosse\AppData\Local\Temp/ipykernel_10864/1784094827.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books90["weighted_rating"] = books90.apply(weighted_rating, axis=1)


In [7]:
books90.sort_values("weighted_rating",ascending=False).head(25)
books90[["book_id","goodreads_book_id","weighted_rating"]].to_csv("data/popularity_books.csv")

In [8]:
books90

,book_id,goodreads_book_id,title,authors,average_rating,ratings_count,weighted_rating
861,862,17332218,"Words of Radiance (The Stormlight Archive, #2)",Brandon Sanderson,4.77,73572,4.495007
421,422,862041,"Harry Potter Boxset (Harry Potter, #1-7)",J.K. Rowling,4.74,190050,4.608935
1307,1308,17927395,A Court of Mist and Fury (A Court of Thorns an...,Sarah J. Maas,4.72,108384,4.522803
2100,2101,99298,"The Harry Potter Collection 1-4 (Harry Potter,...","J.K. Rowling, Mary GrandPré",4.66,43929,4.342224
1009,1010,43070,The Essential Calvin and Hobbes: A Calvin and ...,Bill Watterson,4.65,93001,4.451612
...,...,...,...,...,...,...,...
1865,1866,18641982,The Vacationers,Emma Straub,3.14,44074,3.555799
1546,1547,6976,The Mermaid Chair,Sue Monk Kidd,3.10,62760,3.456775
2135,2136,2152,The Jane Austen Book Club,Karen Joy Fowler,3.07,53128,3.476340
1821,1822,12615008,"Revolution 2020: Love, Corruption, Ambition",Chetan Bhagat,3.07,42845,3.526143
